In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Intro

This notebook runs the neural network models for my masters project, "Feature Extraction
and Machine Learning Techniques for Musical Genre Determination," for which I will be
receiving a Masters of Science in 
[Electrical Engineering](https://www.csun.edu/engineering-computer-science/electrical-computer-engineering/) 
from [California State University, Northridge](https://www.csun.edu/) in December 2017. 
My advisor at CSUN is 
[Dr. Xiyi Hang](https://www.csun.edu/engineering-computer-science/electrical-computer-engineering/xiyi-hang). The accompanying paper is not currently public, but when it is released, a link will be added to this page. Only a partial list of most-relevant references appears in this notebook; the full list appears in the accompanying paper.

In this project, two approaches to musical genre classification were investigated: the use of support vector classification on Mel-frequency cepstral coefficient (MFCC) features (the "[SVCModels.ipynb](SVCModels.ipynb)" notebook), and the use of neural networks on image data generated via the discrete wavelet transform (DWT) (this notebook).

The models used include a shallow two-layer neural network (abbreviated in the code as '`fcnn`', for **f**ully-**c**onnected **n**eural **n**etwork) and four convolutional models ([Inception V3](https://keras.io/applications/#inceptionv3), [Xception](https://keras.io/applications/#xception), [ResNet50](https://keras.io/applications/#resnet50), and [VGG16](https://keras.io/applications/#vgg16)), all instantiated with Keras. For the four convolutional models, the ImageNet weights were loaded at instantiation to speed training. Further training was performed via Google Compute Engine (8 x vCPU, 52 GB memory, 1 x NVIDIA Tesla K80) to bring training time down within reasonable limits.

The dataset used was the [FMA music dataset](https://github.com/mdeff/fma). For neural network processing, the audio in the FMA dataset was first processed by generating time-frequency images from the MP3 audio via the discrete wavelet transform (see "[generate_wavelets.py](generate_wavelets.py)").

## Setup

### Imports

In [13]:
# For nice tables/display helpers:
from IPython.display import display, Audio, HTML
import time # for sleep()
from datetime import datetime

# Tensorflow (for checking GPU):
import tensorflow as tf
from tensorflow.python.client import device_lib

# Keras:
import keras
import keras.applications 

# Math (mostly ceil):
import math
#import numpy as np
import pandas as pd
    
# File/path manipulation:
import os, os.path

# For looking up MAC address (used to uniquify data files):
import uuid

# My utilities:
import custom_keras_utils as cku
import utilities as ut 
import code_timing as timer

# Code to autoreload external modules: http://bit.ly/2wyj7sD
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Check Processor

First, assess whether or not a GPU is available. Because the training runs are so computationally expensive, if a GPU is not available, this notebook will run only very small training runs, to allow development and preliminary testing on a low-powered machine.

In [14]:
devlist = device_lib.list_local_devices()
using_gpu = False
for item in devlist:
    if item.device_type == "CPU":
        print("CPU found...", end = "")
    if item.device_type == "GPU":
        using_gpu = True
        print("GPU found!")
        break
if not using_gpu:
    print("\nNo GPU found; adjusting control variables for quicker training runs.")
    

CPU found...
No GPU found; adjusting control variables for quicker training runs.


### Configuration

The `param_dict` dictionary controls the current run type and parameters, controlling parameters such as the batch size, the epoch size, and the size of the hidden layer in the two-layer network (both on its own and when used as the classifier for the deep networks), as well as several parameters for usability, such as whether or not to display popups when a run completes. Because training runs are time consuming even on a GPU, popups will have the effect of refocusing the notebook if it has been left running in the background while the user performs other tasks. The notebook has the ability to also play audio alerts; to enable these alerts, two .WAV files, "complete.wav" and "error.wav" must be added to the folder in which this notebook is run.

`param_dict` also controls the metadata is recorded in the dataframe that tracks runs: the dataframe will record the time of the run, the MAC address of the machine that ran it, and so forth to allow multiple runs to be easily distinguished in the data analysis phase.

In [18]:
# Default master control variables--these can be modified to change the default run type:
param_dict = {"datasetname": "FMA",
              "resume": False,
              "run_verbosity": 1,            # 0 for silent, 2 for summary, 1 for chatty
              "popup_override": True,       # Manually prevent popups (if this is False,
                                             # they default to the time check in 
                                             # display_popups, below).
              "alert": True,                 # for audio alerts
              "seed": 5,
              "augmentation": 0,             # value to use for data augmentation, [0,1]
              "which_wavelet": "dwt",        # use DWT images instead of CWT images
              "which_size": "small",
              "dataset_size": 6400,          # 6400 training images for the small FMA set
              "hidden_size": 128,
              "source": uuid.getnode(),      # MAC address, to uniquify runs from multiple 
                                             # machines
              "run_crossval": True}

# Suppress popups overnight since no one will be around to be alerted:
param_dict["display_popups"] = lambda: ((datetime.now().hour >= 7 and 
                                         datetime.now().hour < 21) and not
                                         param_dict["popup_override"])



# All models are compiled with the same compile arguments, other than the optimizer:
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
param_dict["run_metadata"] = tf.RunMetadata()
param_dict["compile_args"] = {"loss": 'categorical_crossentropy',
                              "metrics": ['categorical_accuracy'],
                              "options": run_options, # note naming difference!
                              "run_metadata": param_dict["run_metadata"]}

# Set up the GPU-available/CPU-only specific options:
if using_gpu:
    # Large training runs are OK to process on the GPU:
    param_dict["spu"] = "gpu"
    param_dict["pass_epochs"] = 5 #20
    # Adjust so each epoch sees every image about once:
    param_dict["batch_size"] = 128 # 192 is too high for even one epoch of VGG19 on GCE.
                                   # Note: going higher than 64 sometimes will lead to 
                                   # memory issues on VGG16, b/c garbage collection isn't 
                                   # instantaneous and VGG16 has a huge number of 
                                   # parameters, but we want this as large as possible, 
                                   # This problem is ameliorated somewhat by specifying
                                   # a small epoch_batch_size in the call to 
                                   # run_pretrained_model(), which will checkpoint the 
                                   # training every epoch_batch_size epochs to clean up
                                   # memory fragmentation (see also http://bit.ly/2hDHJay )
    param_dict["steps_per_epoch"] = math.ceil(param_dict["dataset_size"]/
                                              param_dict["batch_size"]) 
    param_dict["validation_steps"] = math.ceil(param_dict["dataset_size"]/
                                               (8*param_dict["batch_size"])) 
else:
    # We want to skip them, though, if no GPU is available, while still being able to test
    # the code:
    param_dict["spu"] = "cpu"
    param_dict["pass_epochs"] = 1
    # Adjust for quick development runs, where an "epoch" won't really see the whole 
    # training set:
    param_dict["batch_size"] = 4 # <4 confuses Keras
    param_dict["steps_per_epoch"] = 1
    param_dict["validation_steps"] = math.ceil(16/param_dict["batch_size"]) 
                                              # So we'll check at least 16 examples 
                                              # in validation--much smaller than 
                                              # this confuses Keras.

# Register the names we'll save the data files as, since they get used a bunch of 
# different places:
param_dict["fma_results_name"] = "fma_results_{}".format(param_dict["spu"])
param_dict["crossval_results_name"] = "crossval_results_{}".format(param_dict["spu"])
param_dict["run_timelines_file"] = "run_timelines/timeline_{}_{}.ctf.json".format(
                                        param_dict["spu"],
                                        timer.datetimepath())
param_dict["timing_results_name"] = "timing_results"


# Map from short names to display-friendly names:
param_dict["model_names"] = {}
param_dict["model_names"]["fcnn"] = "Two-Layer Network"
param_dict["model_names"]["xception"] = "Xception"
param_dict["model_names"]["inception_v3"] = "Inception V3"
param_dict["model_names"]["resnet50"] = "ResNet50"
param_dict["model_names"]["vgg16"] = "VGG16"

# Set up path
param_dict["img_dir"] = os.path.join("data",
                             os.path.join("fma_images",
                                  os.path.join("byclass", 
                                       os.path.join(param_dict["which_size"], 
                                                    param_dict["which_wavelet"])
                                      )
                                 )
                            )
print("Using directory {} as source for data.\n".format(param_dict["img_dir"]))

# Configure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

param_dict["classes"] = list(generators["val"].class_indices.keys())
param_dict["num_classes"] = len(param_dict["classes"])

# Set up to track the models created for all runs:
models = {}

# Set up no audio alert mode so that if it's set, the notebook will avoid playing audio 
# without throwing errors:
if not param_dict["alert"]:
    error_file = ""    
    complete_file = ""
else: # Set up for audio alerts--or at least try to...
    complete_file = "complete.wav"
    error_file = "error.wav"
    if not os.path.isfile(complete_file):
        print("\nCould not locate '{}'; disabling audio alerts.".format(complete_file))
        complete_file = ""
        error_file = ""
    elif not os.path.isfile(error_file):
        print("\nCould not locate '{}'; disabling audio alerts.".format(error_file))
        complete_file = ""
        error_file = ""

Using directory data/fma_images/byclass/small/dwt as source for data.

Creating generators with batch size 4...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.

Could not locate 'complete.wav'; disabling audio alerts.


In [23]:
def calc_etas(param_dict):
    param_dict["etas"] = {}
    
    try:
        prev_runs = ut.load_obj(param_dict["fma_results_name"])
    except:
        print("No previous runs found; skipping ETA calculations.")
        
        for model_name in ["fcnn", "xception", "inception_v3", "resnet50", "vgg16"]:
            param_dict["etas"][model_name] = "unknown (no similar runs found)"
            
        return

    # At least some previous runs found; calculate all the ETAS we can:
    grouped_for_etas = prev_runs.groupby(["Source Processor", 
                                          "Pass Epochs", 
                                          "Data Set Size", 
                                          "Model"])["Run Duration"]
    for model_name in ["fcnn", "xception", "inception_v3", "resnet50", "vgg16"]:
        try:
            epochs = param_dict["pass_epochs"]
            eta_run_key = (param_dict["spu"],
                           param_dict["pass_epochs"],
                           param_dict["which_size"],
                           model_name)
            mean_time = (prev_runs.loc[grouped_for_etas.groups[eta_run_key]]
                         ["Run Duration"].mean())
            std_time = (prev_runs.loc[grouped_for_etas.groups[eta_run_key]]
                        ["Run Duration"].std())
            max_time = (prev_runs.loc[grouped_for_etas.groups[eta_run_key]]
                        ["Run Duration"].max())
            min_time = (prev_runs.loc[grouped_for_etas.groups[eta_run_key]]
                        ["Run Duration"].min())
            eta = "{} ± {} ([{},{}])".format(timer.time_from_sec(mean_time),
                                             timer.time_from_sec(std_time),
                                             timer.time_from_sec(min_time),
                                             timer.time_from_sec(max_time))
        except:
            eta = "unknown (no similar runs found)"

        param_dict["etas"][model_name] = eta
        
calc_etas(param_dict)

No previous runs found; skipping ETA calculations.


In [24]:
# Set up some helper functions to consolidate later calls:
js_alert_wrapper = ("<script>var w = window.open('','','width=300,height=200');"
                    "w.document.write('{}');"
                    "w.focus();"
                    "setTimeout(function() {{{{w.close();}}}}, 5000);</script>")
def delayed_popup():
    time.sleep(2)
    if param_dict["display_popups"]():
        display(HTML(js_alert_wrapper.format(js)))

persistent_error = None
def wrap_for_alerts(function_hook, function_name, *args, **kwargs):
    global audio_file
    global js
    global persistent_error
    persistent_error = None
    
    # Only capitalize the first letter without altering other capitalization:
    function_name_init_cap = function_name[0].upper() + function_name[1:]
    try:
        function_hook(*args, **kwargs)

        js = "{} complete!".format(function_name_init_cap)
        audio_file = complete_file
    except Exception as e:
        persistent_error = e
        print(("\nException caught in {}; temporarily handling silently to "
               "allow alerts to go through:").format(function_name))
        display(e)
        js = "{} errored out...".format(function_name_init_cap)
        audio_file = error_file
      
    
# Crossval:
def run_crossval_hook():
    global best_opt_key
    if (param_dict["run_crossval"]):
        best_opt_key = cku.run_crossval(param_dict, opts, models)
        ut.save_obj(best_opt_key, "best_opt_key")
    else:
        best_opt_key = ut.load_obj("best_opt_key")
        print("Best optimizer {} loaded from file.".format(best_opt_key))
        
def run_crossval():
    wrap_for_alerts(run_crossval_hook,"cross-validation")

def model_run_init():
    # Update the ETAs to catch new runs:
    calc_etas(param_dict)
    
    # Load best optimizer to allow running after restart or backend reset:
    param_dict["run_crossval"] = False
    run_crossval() 
    build_opts(best_opt_key)
    
    
# Fully connected network:
def run_fcnn_hook():
    model_run_init()
    
    _ = cku.run_fcnn_model(param_dict, generators, opts, best_opt_key, models, 
                           param_dict["etas"]["fcnn"],
                           epoch_batch_size = 10)
    
def run_fcnn():
    wrap_for_alerts(run_fcnn_hook,"fully connected network run")
    
    
# Xception:
def run_xception_hook():
    # Load best optimizer to allow running after restart or backend reset:
    model_run_init()
    
    cku.run_pretrained_model(param_dict, generators, models, opts, 
                             keras.applications.xception.Xception, "xception", 
                             best_opt_key, False,
                             [122, 105, 95, 85, 75], param_dict["etas"]["xception"],
                             epoch_batch_size = 7)
    
def run_xception():
    wrap_for_alerts(run_xception_hook,"Xception run")
    
    
# InceptionV3:
def run_inception_v3_hook():
    model_run_init()
    
    cku.run_pretrained_model(param_dict, generators, models, opts,
                             keras.applications.inception_v3.InceptionV3, "inception_v3",
                             best_opt_key, False,
                             [249, 232, 229, 200, 187], param_dict["etas"]["inception_v3"],
                             epoch_batch_size = 10)
    
def run_inception_v3():
    wrap_for_alerts(run_inception_v3_hook,"Inception V3 run")
    
    
# ResNet50:
def run_resnet50_hook():
    model_run_init()
    
    cku.run_pretrained_model(param_dict, generators, models, opts, 
                             keras.applications.resnet50.ResNet50, "resnet50",
                             best_opt_key, False, 
                             [161, 151, 139, 129, 119], param_dict["etas"]["resnet50"],
                             epoch_batch_size = 7)
    
def run_resnet50():
    wrap_for_alerts(run_resnet50_hook,"ResNet50 run")
    
    
# VGG16:
def run_vgg16_hook():
    model_run_init()
    
    cku.run_pretrained_model(param_dict, generators, models, opts, 
                             keras.applications.vgg16.VGG16, "vgg16", 
                             best_opt_key, False,
                             [17, 15, 13, 11, 8], param_dict["etas"]["vgg16"],
                             epoch_batch_size = 3)
    
def run_vgg16():
    wrap_for_alerts(run_vgg16_hook,"VGG16 run")
    

In [25]:
def build_opts(opt_key = None):
    global opts
    
    # Set up the optimizers for the cross-validation:
    opts = {}
    if opt_key is None:
        learning_rates = [0.001, 0.005, 0.01]
        rhos = [0.75, 0.8, 0.85, 0.9, 0.95]
        decays = [1e-6, 0.001, 0.01, 0.02, 0.04]
        epsilons = [1e-10, 1e-9, 1e-8, 1e-7]
        for lr in learning_rates:
            for rho in rhos:
                for epsilon in epsilons:
                    for decay in decays:
                        opts[lr, decay, rho, epsilon] = keras.optimizers.RMSprop(lr = lr,
                                                                             decay = decay,
                                                                             rho = rho,
                                                                             epsilon = epsilon)
    else:
        # We really only need the one we've requested:
        opts[opt_key] = keras.optimizers.RMSprop(lr = opt_key[0],
                                                 decay = opt_key[1],
                                                 rho = opt_key[2],
                                                 epsilon = opt_key[3])
        
build_opts()

## Cross-Validation

Before training, quickly evaluate all possible optimizer hyperparameter combinations to find the best combination from the set of hyperparameter values specified. Note that this search is not exhaustive; rather, the best optimizer is simply selected from among a set of plausible values based on a very short (1 epoch) training period.

In [7]:
run_crossval()  
Audio(url=audio_file, autoplay=True)

Cross-validation of 300 optimizers with manual parameters takes about 05:00:00.

Creating generators with batch size 2048...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.

Starting run at Thursday, 2017 October 12, 3:39 PM...1/300 (0.0%)...2/300 (0.3%)...3/300 (0.7%)...4/300 (1.0%)...5/300 (1.3%)...6/300 (1.7%)...7/300 (2.0%)...8/300 (2.3%)...9/300 (2.7%)...10/300 (3.0%)...11/300 (3.3%)...12/300 (3.7%)...13/300 (4.0%)...14/300 (4.3%)...15/300 (4.7%)...16/300 (5.0%)...17/300 (5.3%)...18/300 (5.7%)...19/300 (6.0%)...20/300 (6.3%)...21/300 (6.7%)...22/300 (7.0%)...23/300 (7.3%)...24/300 (7.7%)...25/300 (8.0%)...26/300 (8.3%)...27/300 (8.7%)...28/300 (9.0%)...29/300 (9.3%)...30/300 (9.7%)...31/300 (10.0%)...32/300 (10.3%)...33/300 (10.7%)...34/300 (11.0%)...35/300 (11.3%)...36/300 (11.7%)...37/300 (12.0%).

In [8]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

if persistent_error is not None:
    raise(persistent_error) # crossval is special--other runs depend on it, so halt  
                            # execution if there's an error in crossval

### Results

In [9]:
crossval_results = ut.load_obj(param_dict["crossval_results_name"])
display(crossval_results)

## Simple Two-Layer Neural Network



In [8]:
# Run a full training pass:
run_fcnn()
    
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Monday, 2017 October 16, 10:30 PM.
	[30 epochs on small FMA on GPU takes
	00:09:28 ± 00:04:30 ([00:06:16,00:12:39]).]


Training for epochs 1 to 10...
Epoch 1/10
50/50 [==============================] - 28s - loss: 1.8238 - categorical_accuracy: 0.3172 - val_loss: 1.8220 - val_categorical_accuracy: 0.3638
Epoch 2/10
50/50 [==============================] - 23s - loss: 1.6009 - categorical_accuracy: 0.4188 - val_loss: 1.6651 - val_categorical_accuracy: 0.3688
Epoch 3/10
50/50 [==============================] - 24s - loss: 1.4074 - categorical_accuracy: 0.5067 - val_loss: 1.6592 - val_categorical_accuracy: 0.3638
Epoch 4/10
50/50 [==============================] - 23s - loss: 1.1634 - categorical_accuracy: 0.6016 - val_loss: 1.6317 - val_categorical_accuracy: 0.3950
Epoch 5/10
50/50 [==============================] - 23s - loss: 

In [9]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

Note that the chance accuracy for an 8-genre classification task is 12.5%, so this result exceeds that value significantly. However, the enormous gap between the training accuracy and the validation accuracy indicates that the two-layer neural network has badly overfit the training data.

## Xception

[Xception](https://keras.io/applications/#xception) is a model derived from Inception [1]. It uses the principle of *depthwise separable* convolutions to factorize single large (e.g. 5x5) convolutions into stacks of smaller (e.g. 1x1) convolutions along the depth axis. The Keras instantiation has 22,910,480 parameters across 126 layers and achieves a top-1 accuracy of 79% on ImageNet.


### Relevant Literature

[1] C. Szegedy et al., “Going deeper with convolutions,” Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 1–9.

[2] C. Szegedy et al., “Rethinking the Inception architecture for computer vision,” CoRR, vol.
abs/1512.00567, 2015 [Online]. Available: http://arxiv.org/abs/1512.00567. Last checked: 2017 September 18

[3] F. Chollet, “Xception: Deep learning with depthwise separable convolutions,” CoRR, vol. abs/1610.02357, 2016 [Online]. Available: http://arxiv.org/abs/1610.02357. Last checked: 2017 September 21

In [10]:
run_xception()
    
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Monday, 2017 October 16, 10:42 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 193s - loss: 1.9081 - categorical_accuracy: 0.2856 - val_loss: 2.1487 - val_categorical_accuracy: 0.1625
Epoch 2/5
50/50 [==============================] - 190s - loss: 1.6867 - categorical_accuracy: 0.3852 - val_loss: 2.0583 - val_categorical_accuracy: 0.2512
Epoch 3/5
50/50 [==============================] - 190s - loss: 1.5682 - categorical_accuracy: 0.4344 - val_loss: 1.9656 - val_categorical_accuracy: 0.2888
Epoch 4/5
50/50 [==============================] - 191s - loss: 1.5023 - categorical_accuracy: 0.4545 - val_loss: 1.9731 - val_categorical_accuracy: 0.2687
Epoch 5/5
50/50 [==============================] 

In [11]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

# Inception V3

[Inception V3](https://keras.io/applications/#inceptionv3) is, like Xception, a model derived from Inception [1]. It uses a principle similar to the depthwise separable convolutions upon which Xception derives to likewise improve the efficiency of the original Inception architecture. The Keras instantiation has 23,851,784 parameters across 159 layers and achieves a top-1 accuracy of 78.8% on ImageNet.


### Relevant Literature

[1] C. Szegedy et al., “Going deeper with convolutions,” Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 1–9.

[2] C. Szegedy et al., “Rethinking the Inception architecture for computer vision,” CoRR, vol. abs/1512.00567, 2015 [Online]. Available: http://arxiv.org/abs/1512.00567. Last checked: 2017 September 18


In [12]:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Tuesday, 2017 October 17, 12:40 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 126s - loss: 1.9551 - categorical_accuracy: 0.2591 - val_loss: 4.1225 - val_categorical_accuracy: 0.1237
Epoch 2/5
50/50 [==============================] - 121s - loss: 1.7639 - categorical_accuracy: 0.3445 - val_loss: 2.5452 - val_categorical_accuracy: 0.1700
Epoch 3/5
50/50 [==============================] - 120s - loss: 1.6794 - categorical_accuracy: 0.3853 - val_loss: 2.4868 - val_categorical_accuracy: 0.1925
Epoch 4/5
50/50 [==============================] - 120s - loss: 1.6130 - categorical_accuracy: 0.4133 - val_loss: 2.1394 - val_categorical_accuracy: 0.2537
Epoch 5/5
50/50 [===========================

In [13]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

# ResNet50

[ResNet50](https://keras.io/applications/#resnet50) uses a set of shallow connections (*shortcut connections*) that run alongside the deeper connections in the model to improve training accuracy and allow for easier optimization. The Keras instantiation has 25,636,712 parameters across 168 layers and achieves a top-1 accuracy of 75.9% on ImageNet.

### Relevant Literature

[4] K. He et al., “Deep residual learning for image recognition,” Proceedings of the IEEE conference on computer vision and pattern recognition, pp. 770–778.

[5] C. Szegedy et al., “Inception-v4, Inception-ResNet and the impact of residual connections on learning,” CoRR, vol. abs/1602.07261, 2016 [Online]. Available: http://arxiv.org/abs/1602.07261. Last checked: 2017 October 29

In [14]:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Tuesday, 2017 October 17, 1:53 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 149s - loss: 1.7676 - categorical_accuracy: 0.3503 - val_loss: 3.4586 - val_categorical_accuracy: 0.1713
Epoch 2/5
50/50 [==============================] - 145s - loss: 1.5177 - categorical_accuracy: 0.4537 - val_loss: 2.5427 - val_categorical_accuracy: 0.2525
Epoch 3/5
50/50 [==============================] - 145s - loss: 1.3818 - categorical_accuracy: 0.5127 - val_loss: 2.1963 - val_categorical_accuracy: 0.3187
Epoch 4/5
50/50 [==============================] - 145s - loss: 1.2809 - categorical_accuracy: 0.5400 - val_loss: 1.9553 - val_categorical_accuracy: 0.3738
Epoch 5/5
50/50 [==============================] 

In [15]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

# VGG16

[VGG16](https://keras.io/applications/#vgg16) is the largest, but simplest, convolutional model tested. It is made up of a stack of five blocks, each which consists of some number of convolutional layers followed by a pooling layer. The Keras instantiation has 138,357,544 parameters across 23 layers and achieves a top-1 accuracy of 71.59% on ImageNet.

### Relevant Literature

[6] K. Simonyan and A. Zisserman, “Very deep convolutional networks for large-scale image recognition,” CoRR, vol. abs/1409.1556, 2014 [Online]. Available: http://arxiv.org/abs/1409.1556. Last checked: 2017 October 29

In [16]:
run_vgg16()
    
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Tuesday, 2017 October 17, 3:16 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
50/50 [==============================] - 267s - loss: 1.7977 - categorical_accuracy: 0.3381 - val_loss: 2.8726 - val_categorical_accuracy: 0.1775
Epoch 2/3
50/50 [==============================] - 249s - loss: 1.6469 - categorical_accuracy: 0.4036 - val_loss: 2.1224 - val_categorical_accuracy: 0.2525
Epoch 3/3
50/50 [==============================] - 248s - loss: 1.5977 - categorical_accuracy: 0.4303 - val_loss: 1.7418 - val_categorical_accuracy: 0.3362

Training for epochs 4 to 5...
Epoch 4/5
50/50 [==============================] - 249s - loss: 1.5541 - categorical_accuracy: 0.4447 - val_loss: 1.6696 - val_categorical_accuracy: 0.3625
Epoch 5/5
50/50 [====

In [17]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [20]:
# Back up the results dataframes
import shutil

for key in ["fma_results_name", "crossval_results_name"]:
    src = os.path.join("saved_objects", "{}.pkl".format(param_dict[key])) 
    dst = os.path.join("saved_object_backups", 
                       "{}-{}.pkl.bak".format(param_dict[key],
                                              timer.datetimepath()))
    directory = os.path.dirname(dst)
    if not os.path.exists(directory):
        os.makedirs(directory)
    #shutil.copyfile(src, dst)

    print ("Backed up '{}' to\n\t'{}'.\n".format(src, dst))

Backed up 'saved_objects/fma_results_gpu.pkl' to
	'saved_object_backups/fma_results_gpu-2017-10-17+0927.pkl.bak'.

Backed up 'saved_objects/crossval_results_gpu.pkl' to
	'saved_object_backups/crossval_results_gpu-2017-10-17+0927.pkl.bak'.



# Alternate Approaches to Data

## DWT With Data Augmentation

Because of the small size of the dataset, data augmentation was used to improve results. Because the DWT images take a long time to generate, this augmentation was performed at the image level, but only those forms of image-based augmentation that are meaningful to perform on audio were used--i.e., only horizontal shift, which performs an operation similar to windowing. 

See Section 4.5 of the accompanying paper, "Experiment 4: Small Dataset With Augmentation,"  for more details, as well as Section 5.6, "Approaches to Data Augmentation," for the proposal of an alternate approach that may be more appropriate for audio sources.

### Section-Specific Setup

In [8]:
# Enable data augmentation:
param_dict["augmentation"] = 0.66 # i.e., keep at least 10 sec of 30 sec clips
# Reconfigure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

Creating generators with batch size 128...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Using up to 66.0% horizontal shift to augment training data.
Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


### Model Reruns

In [ ]:
# Now rerun all 5 models, starting with FCNN:
run_fcnn()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Tuesday, 2017 October 17, 9:27 AM.
	[30 epochs on small FMA on GPU takes
	00:10:35 ± 00:03:44 ([00:06:16,00:12:50]).]


Training for epochs 1 to 10...
Epoch 1/10
50/50 [==============================] - 96s - loss: 1.8237 - categorical_accuracy: 0.3123 - val_loss: 1.8345 - val_categorical_accuracy: 0.3513
Epoch 2/10
50/50 [==============================] - 89s - loss: 1.7128 - categorical_accuracy: 0.3717 - val_loss: 1.7082 - val_categorical_accuracy: 0.3775
Epoch 3/10
50/50 [==============================] - 87s - loss: 1.6821 - categorical_accuracy: 0.3845 - val_loss: 1.6368 - val_categorical_accuracy: 0.3837
Epoch 4/10
50/50 [==============================] - 88s - loss: 1.6529 - categorical_accuracy: 0.4017 - val_loss: 1.6114 - val_categorical_accuracy: 0.4050
Epoch 5/10
50/50 [==============================] - 88s - loss: 

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [ ]:
# Xception:
run_xception()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Tuesday, 2017 October 17, 10:13 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:23:31 ± 00:48:24 ([00:49:18,01:57:45]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 194s - loss: 1.9076 - categorical_accuracy: 0.2753 - val_loss: 2.1800 - val_categorical_accuracy: 0.1663
Epoch 2/5
50/50 [==============================] - 191s - loss: 1.7660 - categorical_accuracy: 0.3481 - val_loss: 2.0781 - val_categorical_accuracy: 0.2087
Epoch 3/5
50/50 [==============================] - 192s - loss: 1.7340 - categorical_accuracy: 0.3603 - val_loss: 2.0065 - val_categorical_accuracy: 0.2637
Epoch 4/5
50/50 [==============================] - 192s - loss: 1.6890 - categorical_accuracy: 0.3717 - val_loss: 1.9344 - val_categorical_accuracy: 0.2925
Epoch 5/5
50/50 [=====================

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [ ]:
# InceptionV3:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Tuesday, 2017 October 17, 12:11 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 126s - loss: 1.9564 - categorical_accuracy: 0.2580 - val_loss: 3.3636 - val_categorical_accuracy: 0.1250
Epoch 2/5
50/50 [==============================] - 122s - loss: 1.8535 - categorical_accuracy: 0.3005 - val_loss: 2.6411 - val_categorical_accuracy: 0.1737
Epoch 3/5
50/50 [==============================] - 122s - loss: 1.8162 - categorical_accuracy: 0.3152 - val_loss: 2.6110 - val_categorical_accuracy: 0.1825
Epoch 4/5
50/50 [==============================] - 122s - loss: 1.7987 - categorical_accuracy: 0.3277 - val_loss: 2.1234 - val_categorical_accuracy: 0.2313
Epoch 5/5
50/50 [===========================

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [ ]:
# ResNet50:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Tuesday, 2017 October 17, 1:25 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 150s - loss: 1.7666 - categorical_accuracy: 0.3488 - val_loss: 3.1254 - val_categorical_accuracy: 0.1825
Epoch 2/5
50/50 [==============================] - 147s - loss: 1.6133 - categorical_accuracy: 0.4153 - val_loss: 2.3868 - val_categorical_accuracy: 0.2575
Epoch 3/5
50/50 [==============================] - 148s - loss: 1.5702 - categorical_accuracy: 0.4256 - val_loss: 2.0260 - val_categorical_accuracy: 0.3075
Epoch 4/5
50/50 [==============================] - 148s - loss: 1.5362 - categorical_accuracy: 0.4394 - val_loss: 1.8820 - val_categorical_accuracy: 0.3513
Epoch 5/5
50/50 [==============================] 

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [ ]:
# VGG16:
run_vgg16()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Tuesday, 2017 October 17, 2:49 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
50/50 [==============================] - 251s - loss: 1.8052 - categorical_accuracy: 0.3428 - val_loss: 3.0211 - val_categorical_accuracy: 0.1825
Epoch 2/3
50/50 [==============================] - 249s - loss: 1.6764 - categorical_accuracy: 0.3912 - val_loss: 1.8996 - val_categorical_accuracy: 0.2975
Epoch 3/3
50/50 [==============================] - 249s - loss: 1.6254 - categorical_accuracy: 0.4081 - val_loss: 1.7037 - val_categorical_accuracy: 0.3700

Training for epochs 4 to 5...
Epoch 4/5
50/50 [==============================] - 251s - loss: 1.5762 - categorical_accuracy: 0.4281 - val_loss: 1.6391 - val_categorical_accuracy: 0.3850
Epoch 5/5
50/50 [====

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

## Varying the Value of the Shift

### Section-Specific Setup (50% Augmentation)

In [10]:
# Alter data augmentation value:
param_dict["augmentation"] = 0.5 # i.e., keep at least 15 sec of 30 sec clips
# Reconfigure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

Creating generators with batch size 128...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Using up to 50.0% horizontal shift to augment training data.
Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


### Model Reruns

In [12]:
# FCNN:
run_fcnn()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Tuesday, 2017 October 17, 11:16 PM.
	[30 epochs on small FMA on GPU takes
	00:19:18 ± 00:17:42 ([00:06:16,00:45:27]).]


Training for epochs 1 to 10...
Epoch 1/10
50/50 [==============================] - 95s - loss: 1.8272 - categorical_accuracy: 0.3162 - val_loss: 1.8042 - val_categorical_accuracy: 0.3613
Epoch 2/10
50/50 [==============================] - 91s - loss: 1.7175 - categorical_accuracy: 0.3644 - val_loss: 1.6865 - val_categorical_accuracy: 0.3837
Epoch 3/10
50/50 [==============================] - 91s - loss: 1.6722 - categorical_accuracy: 0.3853 - val_loss: 1.6498 - val_categorical_accuracy: 0.3750
Epoch 4/10
50/50 [==============================] - 88s - loss: 1.6570 - categorical_accuracy: 0.3964 - val_loss: 1.6056 - val_categorical_accuracy: 0.4025
Epoch 5/10
50/50 [==============================] - 86s - loss:

In [13]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [14]:
# Xception:
run_xception()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Wednesday, 2017 October 18, 12:01 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:35:10 ± 00:39:44 ([00:49:18,01:58:29]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 193s - loss: 1.9261 - categorical_accuracy: 0.2717 - val_loss: 2.3211 - val_categorical_accuracy: 0.2100
Epoch 2/5
50/50 [==============================] - 190s - loss: 1.7864 - categorical_accuracy: 0.3294 - val_loss: 2.7821 - val_categorical_accuracy: 0.1375
Epoch 3/5
50/50 [==============================] - 190s - loss: 1.7452 - categorical_accuracy: 0.3478 - val_loss: 1.9734 - val_categorical_accuracy: 0.2250
Epoch 4/5
50/50 [==============================] - 190s - loss: 1.7187 - categorical_accuracy: 0.3575 - val_loss: 1.9125 - val_categorical_accuracy: 0.2825
Epoch 5/5
50/50 [===================

In [15]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [16]:
# Inception V3:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Wednesday, 2017 October 18, 1:59 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:12:29 ± 47.949 sec ([01:11:55,01:13:03]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 127s - loss: 1.9566 - categorical_accuracy: 0.2469 - val_loss: 4.4766 - val_categorical_accuracy: 0.1263
Epoch 2/5
50/50 [==============================] - 121s - loss: 1.8499 - categorical_accuracy: 0.3014 - val_loss: 2.7545 - val_categorical_accuracy: 0.1525
Epoch 3/5
50/50 [==============================] - 121s - loss: 1.8171 - categorical_accuracy: 0.3233 - val_loss: 2.3754 - val_categorical_accuracy: 0.1800
Epoch 4/5
50/50 [==============================] - 121s - loss: 1.7820 - categorical_accuracy: 0.3308 - val_loss: 2.1441 - val_categorical_accuracy: 0.2238
Epoch 5/5
50/50 [==============

In [17]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [18]:
# ResNet50:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Wednesday, 2017 October 18, 3:11 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:23:47 ± 46.411 sec ([01:23:14,01:24:20]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 151s - loss: 1.7646 - categorical_accuracy: 0.3555 - val_loss: 3.9853 - val_categorical_accuracy: 0.1363
Epoch 2/5
50/50 [==============================] - 145s - loss: 1.6147 - categorical_accuracy: 0.4095 - val_loss: 3.0745 - val_categorical_accuracy: 0.2625
Epoch 3/5
50/50 [==============================] - 145s - loss: 1.5612 - categorical_accuracy: 0.4270 - val_loss: 2.2987 - val_categorical_accuracy: 0.3025
Epoch 4/5
50/50 [==============================] - 145s - loss: 1.5421 - categorical_accuracy: 0.4416 - val_loss: 1.8570 - val_categorical_accuracy: 0.3525
Epoch 5/5
50/50 [==================

In [19]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [11]:
# VGG16:
run_vgg16()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Friday, 2017 October 20, 8:20 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	02:44:48 ± 15.410 sec ([02:44:37,02:44:59]).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
50/50 [==============================] - 259s - loss: 1.8070 - categorical_accuracy: 0.3317 - val_loss: 2.9281 - val_categorical_accuracy: 0.2175
Epoch 2/3
50/50 [==============================] - 239s - loss: 1.6816 - categorical_accuracy: 0.3877 - val_loss: 1.8851 - val_categorical_accuracy: 0.2888
Epoch 3/3
50/50 [==============================] - 240s - loss: 1.6257 - categorical_accuracy: 0.4053 - val_loss: 1.8181 - val_categorical_accuracy: 0.2875

Training for epochs 4 to 5...
Epoch 4/5
50/50 [==============================] - 244s - loss: 1.5842 - categorical_accuracy: 0.4228 - val_loss: 1.6720 - val_categorical_accuracy: 0.3775
Epoch 5/5


In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

### Section-Specific Setup (33% Augmentation)

In [12]:
# Alter data augmentation value:
param_dict["augmentation"] = 0.33 # i.e., keep at least 20 sec of 30 sec clips
# Reconfigure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

Creating generators with batch size 128...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Using up to 33.0% horizontal shift to augment training data.
Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


### Model Reruns

In [21]:
# FCNN:
run_fcnn()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Wednesday, 2017 October 18, 4:35 AM.
	[30 epochs on small FMA on GPU takes
	00:24:26 ± 00:19:09 ([00:06:16,00:45:27]).]


Training for epochs 1 to 10...
Epoch 1/10
50/50 [==============================] - 97s - loss: 1.8202 - categorical_accuracy: 0.3175 - val_loss: 1.8147 - val_categorical_accuracy: 0.3500
Epoch 2/10
50/50 [==============================] - 88s - loss: 1.7107 - categorical_accuracy: 0.3711 - val_loss: 1.6795 - val_categorical_accuracy: 0.3750
Epoch 3/10
50/50 [==============================] - 89s - loss: 1.6714 - categorical_accuracy: 0.3883 - val_loss: 1.6312 - val_categorical_accuracy: 0.3900
Epoch 4/10
50/50 [==============================] - 87s - loss: 1.6512 - categorical_accuracy: 0.4058 - val_loss: 1.6011 - val_categorical_accuracy: 0.4050
Epoch 5/10
50/50 [==============================] - 87s - loss

In [22]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [23]:
# Xception:
run_xception()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Wednesday, 2017 October 18, 5:20 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:40:47 ± 00:34:20 ([00:49:18,01:58:29]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 194s - loss: 1.9206 - categorical_accuracy: 0.2734 - val_loss: 2.2645 - val_categorical_accuracy: 0.1525
Epoch 2/5
50/50 [==============================] - 191s - loss: 1.7789 - categorical_accuracy: 0.3384 - val_loss: 2.1427 - val_categorical_accuracy: 0.2450
Epoch 3/5
50/50 [==============================] - 191s - loss: 1.7419 - categorical_accuracy: 0.3519 - val_loss: 2.0722 - val_categorical_accuracy: 0.2137
Epoch 4/5
50/50 [==============================] - 191s - loss: 1.7101 - categorical_accuracy: 0.3684 - val_loss: 1.8965 - val_categorical_accuracy: 0.3088
Epoch 5/5
50/50 [====================

In [24]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [25]:
# Inception V3:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Wednesday, 2017 October 18, 7:18 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:12:17 ± 39.542 sec ([01:11:53,01:13:03]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 126s - loss: 1.9689 - categorical_accuracy: 0.2400 - val_loss: 2.5845 - val_categorical_accuracy: 0.1437
Epoch 2/5
50/50 [==============================] - 123s - loss: 1.8507 - categorical_accuracy: 0.3053 - val_loss: 2.3247 - val_categorical_accuracy: 0.1737
Epoch 3/5
50/50 [==============================] - 123s - loss: 1.8323 - categorical_accuracy: 0.3086 - val_loss: 2.2459 - val_categorical_accuracy: 0.2225
Epoch 4/5
50/50 [==============================] - 122s - loss: 1.7882 - categorical_accuracy: 0.3358 - val_loss: 2.1756 - val_categorical_accuracy: 0.2263
Epoch 5/5
50/50 [==============

In [26]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [27]:
# ResNet50:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Wednesday, 2017 October 18, 8:32 AM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	01:23:34 ± 39.646 sec ([01:23:09,01:24:20]).]

First-round training (training the classifier)...

Training for epochs 1 to 5...
Epoch 1/5
50/50 [==============================] - 150s - loss: 1.7686 - categorical_accuracy: 0.3448 - val_loss: 3.1638 - val_categorical_accuracy: 0.1425
Epoch 2/5
50/50 [==============================] - 147s - loss: 1.6120 - categorical_accuracy: 0.4164 - val_loss: 2.7485 - val_categorical_accuracy: 0.2462
Epoch 3/5
50/50 [==============================] - 146s - loss: 1.5702 - categorical_accuracy: 0.4306 - val_loss: 2.2845 - val_categorical_accuracy: 0.3075
Epoch 4/5
50/50 [==============================] - 147s - loss: 1.5422 - categorical_accuracy: 0.4380 - val_loss: 1.9024 - val_categorical_accuracy: 0.3425
Epoch 5/5
50/50 [==================

In [28]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [13]:
# VGG16:
run_vgg16()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Friday, 2017 October 20, 11:02 PM.
	[5 epochs (x6 passes) on small FMA on GPU takes
	02:44:00 ± 00:01:25 ([02:42:22,02:44:59]).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
50/50 [==============================] - 248s - loss: 1.8049 - categorical_accuracy: 0.3383 - val_loss: 2.6464 - val_categorical_accuracy: 0.1925
Epoch 2/3
50/50 [==============================] - 247s - loss: 1.6688 - categorical_accuracy: 0.3934 - val_loss: 1.8562 - val_categorical_accuracy: 0.3088
Epoch 3/3
50/50 [==============================] - 247s - loss: 1.6295 - categorical_accuracy: 0.4138 - val_loss: 1.7421 - val_categorical_accuracy: 0.3350

Training for epochs 4 to 5...
Epoch 4/5
50/50 [==============================] - 249s - loss: 1.5873 - categorical_accuracy: 0.4312 - val_loss: 1.6919 - val_categorical_accuracy: 0.3850
Epoch 5/5
5

In [14]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [15]:
# Back up the results dataframes
import shutil

for key in ["fma_results_name", "crossval_results_name"]:
    src = os.path.join("saved_objects", "{}.pkl".format(param_dict[key])) 
    dst = os.path.join("saved_object_backups", 
                       "{}-{}.pkl.bak".format(param_dict[key],
                                              timer.datetimepath()))
    directory = os.path.dirname(dst)
    if not os.path.exists(directory):
        os.makedirs(directory)
    #shutil.copyfile(src, dst)

    print ("Backed up '{}' to\n\t'{}'.\n".format(src, dst))

Backed up 'saved_objects/fma_results_gpu.pkl' to
	'saved_object_backups/fma_results_gpu-2017-10-21+0144.pkl.bak'.

Backed up 'saved_objects/crossval_results_gpu.pkl' to
	'saved_object_backups/crossval_results_gpu-2017-10-21+0144.pkl.bak'.



# A Larger Dataset

The **FMA extended** dataset expands the 8-genre, "small" subset included in the FMA project by gathering all audio tracks in those 8 genres from the "large" subset included in the FMA project. This yields a total dataset size of 37,316 tracks, versus the 8,000 tracks in the **FMA small** dataset. However, unlike the FMA small dataset used earlier, the FMA extended dataset is not balanced by genre, i.e., there are many more examples of some genres, like rock, than there are of other genres, like folk. Unbalanced datasets are known to be a potential source of problems in training.

For more information, see Section 4.6 of the accompanying paper, "Experiment 5: Extended Dataset."

### Section-Specific Setup

In [8]:
param_dict["augmentation"] = 0 # Start with no data augmentation
param_dict["which_size"] = "extended" 

real_dataset_size = 37316 # There are 37316 training images in the data set
epoch_ratio = 1/4 # This is to keep the number of steps per epoch under control, especially
                  # for large networks like VGG16/VGG19. It necessitates multiplying the 
                  # number of epochs by the inverse to get an accurate number of "real"
                  # epochs.
param_dict["dataset_size"] = math.ceil(epoch_ratio*real_dataset_size) # Round up

# Reconfigure the GPU-available/CPU-only specific options:
if using_gpu:
    # Large training runs are OK to process on the GPU:
    param_dict["spu"] = "gpu" 
    param_dict["pass_epochs"] = math.ceil(5/epoch_ratio) # Run for more "epochs" to make up
                                                         # for dividing epochs into parts
    # Adjust so each epoch sees every image about once:
    param_dict["batch_size"] = 128 # 192 is too high for even one epoch of VGG19 on GCE.
                                   # Note: going higher than 64 sometimes will lead to 
                                   # memory issues on VGG16, b/c garbage collection isn't 
                                   # instantaneous and VGG16 has a huge number of 
                                   # parameters, but we want this as large as possible, 
                                   # This problem is ameliorated somewhat by specifying
                                   # a small epoch_batch_size in the call to 
                                   # run_pretrained_model(), which will checkpoint the 
                                   # training every epoch_batch_size epochs to clean up
                                   # memory fragmentation (see also http://bit.ly/2hDHJay )
    param_dict["steps_per_epoch"] = math.ceil(param_dict["dataset_size"]/
                                              param_dict["batch_size"]) 
    param_dict["validation_steps"] = math.ceil(param_dict["dataset_size"]/
                                               (8*param_dict["batch_size"]))  

# Update the path for images:
param_dict["img_dir"] = os.path.join("data",
                             os.path.join("fma_images",
                                  os.path.join("byclass", 
                                       os.path.join(param_dict["which_size"], 
                                                    param_dict["which_wavelet"])
                                      )
                                 )
                            )
    
# Update ETAs:
calc_etas(param_dict)

# Reconfigure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

Creating generators with batch size 128...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_extended_dwt_stats.npz'.

Found 37316 images belonging to 8 classes.
Found 4350 images belonging to 8 classes.
Found 4651 images belonging to 8 classes.


### Model Reruns

In [36]:
# Now rerun all 5 models, starting with FCNN:
run_fcnn()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Wednesday, 2017 October 18, 11:51 AM.
	[120 epochs on extended FMA on GPU takes
	unknown (no similar runs found).]


Training for epochs 1 to 10...
Epoch 1/10
73/73 [==============================] - 40s - loss: 1.5859 - categorical_accuracy: 0.4341 - val_loss: 1.5664 - val_categorical_accuracy: 0.4914
Epoch 2/10
73/73 [==============================] - 39s - loss: 1.4681 - categorical_accuracy: 0.4942 - val_loss: 1.4673 - val_categorical_accuracy: 0.5000
Epoch 3/10
73/73 [==============================] - 40s - loss: 1.4401 - categorical_accuracy: 0.4973 - val_loss: 1.4393 - val_categorical_accuracy: 0.5062
Epoch 4/10
73/73 [==============================] - 39s - loss: 1.4341 - categorical_accuracy: 0.4953 - val_loss: 1.4281 - val_categorical_accuracy: 0.4922
Epoch 5/10
73/73 [==============================] - 39s - loss: 1.3

Epoch 101/110
73/73 [==============================] - 41s - loss: 0.8714 - categorical_accuracy: 0.7029 - val_loss: 1.6303 - val_categorical_accuracy: 0.4992
Epoch 102/110
73/73 [==============================] - 40s - loss: 0.8434 - categorical_accuracy: 0.7143 - val_loss: 1.6408 - val_categorical_accuracy: 0.5055
Epoch 103/110
73/73 [==============================] - 39s - loss: 0.8921 - categorical_accuracy: 0.6925 - val_loss: 1.6366 - val_categorical_accuracy: 0.5023
Epoch 104/110
73/73 [==============================] - 39s - loss: 0.9264 - categorical_accuracy: 0.6818 - val_loss: 1.6445 - val_categorical_accuracy: 0.4992
Epoch 105/110
73/73 [==============================] - 39s - loss: 0.8718 - categorical_accuracy: 0.6990 - val_loss: 1.6486 - val_categorical_accuracy: 0.4969
Epoch 106/110
73/73 [==============================] - 40s - loss: 0.8705 - categorical_accuracy: 0.7025 - val_loss: 1.6399 - val_categorical_accuracy: 0.5039
Epoch 107/110
73/73 [=========================

In [ ]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [17]:
# Xception:
run_xception()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Saturday, 2017 October 21, 1:44 AM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 7...
Epoch 1/7
73/73 [==============================] - 281s - loss: 1.6094 - categorical_accuracy: 0.4272 - val_loss: 1.7768 - val_categorical_accuracy: 0.3867
Epoch 2/7
73/73 [==============================] - 281s - loss: 1.5034 - categorical_accuracy: 0.4721 - val_loss: 1.7590 - val_categorical_accuracy: 0.4188
Epoch 3/7
73/73 [==============================] - 281s - loss: 1.4911 - categorical_accuracy: 0.4714 - val_loss: 1.6319 - val_categorical_accuracy: 0.4320
Epoch 4/7
73/73 [==============================] - 281s - loss: 1.4762 - categorical_accuracy: 0.4802 - val_loss: 1.5202 - val_categorical_accuracy: 0.4914
Epoch 5/7
73/73 [===========================

73/73 [==============================] - 386s - loss: 1.2316 - categorical_accuracy: 0.5693 - val_loss: 1.4237 - val_categorical_accuracy: 0.5164
Epoch 98/100
73/73 [==============================] - 385s - loss: 1.2457 - categorical_accuracy: 0.5638 - val_loss: 1.4265 - val_categorical_accuracy: 0.5172
Epoch 99/100
73/73 [==============================] - 387s - loss: 1.2043 - categorical_accuracy: 0.5750 - val_loss: 1.4236 - val_categorical_accuracy: 0.5141
Epoch 100/100
73/73 [==============================] - 388s - loss: 1.2155 - categorical_accuracy: 0.5747 - val_loss: 1.4229 - val_categorical_accuracy: 0.5156


Further training (refining convolutional blocks, starting with
	layer 75)...

Training for epochs 101 to 107...
Epoch 101/107
73/73 [==============================] - 411s - loss: 1.2226 - categorical_accuracy: 0.5716 - val_loss: 1.4228 - val_categorical_accuracy: 0.5148
Epoch 102/107
73/73 [==============================] - 411s - loss: 1.2079 - categorical_accuracy: 0.5

In [18]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [38]:
# InceptionV3:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Wednesday, 2017 October 18, 1:33 PM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 10...
Epoch 1/10
73/73 [==============================] - 176s - loss: 1.6641 - categorical_accuracy: 0.4037 - val_loss: 3.6289 - val_categorical_accuracy: 0.1938
Epoch 2/10
73/73 [==============================] - 174s - loss: 1.5677 - categorical_accuracy: 0.4429 - val_loss: 2.2224 - val_categorical_accuracy: 0.2102
Epoch 3/10
73/73 [==============================] - 174s - loss: 1.5564 - categorical_accuracy: 0.4467 - val_loss: 1.6468 - val_categorical_accuracy: 0.4266
Epoch 4/10
73/73 [==============================] - 175s - loss: 1.5341 - categorical_accuracy: 0.4494 - val_loss: 1.5183 - val_categorical_accuracy: 0.4695
Epoch 5/10
73/73 [================

73/73 [==============================] - 231s - loss: 1.1561 - categorical_accuracy: 0.5899 - val_loss: 1.4359 - val_categorical_accuracy: 0.4992
Epoch 98/100
73/73 [==============================] - 229s - loss: 1.1564 - categorical_accuracy: 0.5989 - val_loss: 1.4355 - val_categorical_accuracy: 0.5023
Epoch 99/100
73/73 [==============================] - 231s - loss: 1.1069 - categorical_accuracy: 0.6113 - val_loss: 1.4348 - val_categorical_accuracy: 0.5031
Epoch 100/100
73/73 [==============================] - 232s - loss: 1.1158 - categorical_accuracy: 0.6128 - val_loss: 1.4374 - val_categorical_accuracy: 0.5023


Further training (refining convolutional blocks, starting with
	layer 187)...

Training for epochs 101 to 110...
Epoch 101/110
73/73 [==============================] - 242s - loss: 1.1076 - categorical_accuracy: 0.6123 - val_loss: 1.4377 - val_categorical_accuracy: 0.5039
Epoch 102/110
73/73 [==============================] - 238s - loss: 1.0882 - categorical_accuracy: 0.

In [39]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [40]:
# ResNet50:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Wednesday, 2017 October 18, 8:36 PM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 7...
Epoch 1/7
73/73 [==============================] - 221s - loss: 1.5158 - categorical_accuracy: 0.4727 - val_loss: 1.6949 - val_categorical_accuracy: 0.4422
Epoch 2/7
73/73 [==============================] - 218s - loss: 1.4046 - categorical_accuracy: 0.5134 - val_loss: 1.5776 - val_categorical_accuracy: 0.4414
Epoch 3/7
73/73 [==============================] - 217s - loss: 1.3760 - categorical_accuracy: 0.5222 - val_loss: 1.4455 - val_categorical_accuracy: 0.5070
Epoch 4/7
73/73 [==============================] - 218s - loss: 1.3786 - categorical_accuracy: 0.5180 - val_loss: 1.3972 - val_categorical_accuracy: 0.5492
Epoch 5/7
73/73 [==========================

73/73 [==============================] - 272s - loss: 1.0499 - categorical_accuracy: 0.6387 - val_loss: 1.3664 - val_categorical_accuracy: 0.5320
Epoch 98/100
73/73 [==============================] - 271s - loss: 1.0697 - categorical_accuracy: 0.6323 - val_loss: 1.3669 - val_categorical_accuracy: 0.5297
Epoch 99/100
73/73 [==============================] - 271s - loss: 1.0101 - categorical_accuracy: 0.6514 - val_loss: 1.3684 - val_categorical_accuracy: 0.5305
Epoch 100/100
73/73 [==============================] - 273s - loss: 0.9987 - categorical_accuracy: 0.6565 - val_loss: 1.3684 - val_categorical_accuracy: 0.5281


Further training (refining convolutional blocks, starting with
	layer 119)...

Training for epochs 101 to 107...
Epoch 101/107
73/73 [==============================] - 288s - loss: 1.0243 - categorical_accuracy: 0.6459 - val_loss: 1.3666 - val_categorical_accuracy: 0.5297
Epoch 102/107
73/73 [==============================] - 283s - loss: 1.0090 - categorical_accuracy: 0.

In [41]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [9]:
# VGG16:
run_vgg16()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Saturday, 2017 October 21, 1:41 PM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
73/73 [==============================] - 367s - loss: 1.5393 - categorical_accuracy: 0.4618 - val_loss: 1.9379 - val_categorical_accuracy: 0.4820
Epoch 2/3
73/73 [==============================] - 354s - loss: 1.4202 - categorical_accuracy: 0.5056 - val_loss: 1.6293 - val_categorical_accuracy: 0.4062
Epoch 3/3
73/73 [==============================] - 358s - loss: 1.3949 - categorical_accuracy: 0.5205 - val_loss: 1.4364 - val_categorical_accuracy: 0.4992

Training for epochs 4 to 6...
Epoch 4/6
73/73 [==============================] - 366s - loss: 1.3942 - categorical_accuracy: 0.5196 - val_loss: 1.4910 - val_categorical_accuracy: 0.4883
Epoch 5/6
73/73 

73/73 [==============================] - 567s - loss: 0.9914 - categorical_accuracy: 0.6663 - val_loss: 1.3479 - val_categorical_accuracy: 0.5445
Epoch 94/95
73/73 [==============================] - 566s - loss: 0.9632 - categorical_accuracy: 0.6736 - val_loss: 1.3339 - val_categorical_accuracy: 0.5508
Epoch 95/95
73/73 [==============================] - 567s - loss: 1.0041 - categorical_accuracy: 0.6550 - val_loss: 1.3064 - val_categorical_accuracy: 0.5687

Training for epochs 96 to 98...
Epoch 96/98
73/73 [==============================] - 567s - loss: 0.9732 - categorical_accuracy: 0.6729 - val_loss: 1.3797 - val_categorical_accuracy: 0.5477
Epoch 97/98
73/73 [==============================] - 566s - loss: 0.9456 - categorical_accuracy: 0.6771 - val_loss: 1.3242 - val_categorical_accuracy: 0.5500
Epoch 98/98
73/73 [==============================] - 567s - loss: 0.9834 - categorical_accuracy: 0.6614 - val_loss: 1.4140 - val_categorical_accuracy: 0.5258

Training for epochs 99 to 100.

In [10]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

## With Augmentation

### Section-Specific Setup

In [11]:
param_dict["augmentation"] = 0.33 
param_dict["which_size"] = "extended" 

real_dataset_size = 37316 # There are 37316 training images in the data set
epoch_ratio = 1/4 # This is to keep the number of steps per epoch under control, especially
                  # for large networks like VGG16/VGG19. It necessitates multiplying the 
                  # number of epochs by the inverse to get an accurate number of "real"
                  # epochs.
param_dict["dataset_size"] = math.ceil(epoch_ratio*real_dataset_size) # Round up

# Reconfigure the GPU-available/CPU-only specific options:
if using_gpu:
    # Large training runs are OK to process on the GPU:
    param_dict["spu"] = "gpu"
    param_dict["pass_epochs"] = math.ceil(5/epoch_ratio) # Run for more "epochs" to make up
                                                         # for dividing epochs into parts
    # Adjust so each epoch sees every image about once:
    param_dict["batch_size"] = 128 # 192 is too high for even one epoch of VGG19 on GCE.
                                   # Note: going higher than 64 sometimes will lead to 
                                   # memory issues on VGG16, b/c garbage collection isn't 
                                   # instantaneous and VGG16 has a huge number of 
                                   # parameters, but we want this as large as possible, 
                                   # This problem is ameliorated somewhat by specifying
                                   # a small epoch_batch_size in the call to 
                                   # run_pretrained_model(), which will checkpoint the 
                                   # training every epoch_batch_size epochs to clean up
                                   # memory fragmentation (see also http://bit.ly/2hDHJay )
    param_dict["steps_per_epoch"] = math.ceil(param_dict["dataset_size"]/
                                              param_dict["batch_size"]) 
    param_dict["validation_steps"] = math.ceil(param_dict["dataset_size"]/
                                               (8*param_dict["batch_size"])) 
    
# Update the path for images:
param_dict["img_dir"] = os.path.join("data",
                             os.path.join("fma_images",
                                  os.path.join("byclass", 
                                       os.path.join(param_dict["which_size"], 
                                                    param_dict["which_wavelet"])
                                      )
                                 )
                            )
    
# Update ETAs:
calc_etas(param_dict)

# Reconfigure the generators based on the specified parameters:
generators = {}
(generators["train"], 
    generators["val"], 
    generators["test"]) = cku.set_up_generators(param_dict)

Creating generators with batch size 128...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_extended_dwt_stats.npz'.

Using up to 33.0% horizontal shift to augment training data.
Found 37316 images belonging to 8 classes.
Found 4350 images belonging to 8 classes.
Found 4651 images belonging to 8 classes.


### Model Reruns

In [17]:
# Now rerun all 5 models, starting with FCNN:
run_fcnn()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using hidden size 128 and optimizer (0.01, 0.02, 0.85, 1e-10)...
Fully connected network run begun at Thursday, 2017 October 19, 12:47 PM.
	[120 epochs on extended FMA on GPU takes
	unknown (no similar runs found).]


Training for epochs 1 to 10...
Epoch 1/10
73/73 [==============================] - 135s - loss: 1.5870 - categorical_accuracy: 0.4331 - val_loss: 1.5440 - val_categorical_accuracy: 0.4945
Epoch 2/10
73/73 [==============================] - 129s - loss: 1.4728 - categorical_accuracy: 0.4890 - val_loss: 1.4833 - val_categorical_accuracy: 0.4898
Epoch 3/10
73/73 [==============================] - 129s - loss: 1.4476 - categorical_accuracy: 0.4892 - val_loss: 1.4524 - val_categorical_accuracy: 0.4969
Epoch 4/10
73/73 [==============================] - 128s - loss: 1.4422 - categorical_accuracy: 0.4966 - val_loss: 1.4277 - val_categorical_accuracy: 0.4938
Epoch 5/10
73/73 [==============================] - 128s - loss:

73/73 [==============================] - 127s - loss: 1.3044 - categorical_accuracy: 0.5515 - val_loss: 1.3872 - val_categorical_accuracy: 0.5383

Training for epochs 101 to 110...
Epoch 101/110
73/73 [==============================] - 134s - loss: 1.2217 - categorical_accuracy: 0.5771 - val_loss: 1.3857 - val_categorical_accuracy: 0.5406
Epoch 102/110
73/73 [==============================] - 130s - loss: 1.1861 - categorical_accuracy: 0.5947 - val_loss: 1.3920 - val_categorical_accuracy: 0.5320
Epoch 103/110
73/73 [==============================] - 130s - loss: 1.2125 - categorical_accuracy: 0.5761 - val_loss: 1.3931 - val_categorical_accuracy: 0.5344
Epoch 104/110
73/73 [==============================] - 129s - loss: 1.2451 - categorical_accuracy: 0.5723 - val_loss: 1.3943 - val_categorical_accuracy: 0.5312
Epoch 105/110
73/73 [==============================] - 129s - loss: 1.2536 - categorical_accuracy: 0.5628 - val_loss: 1.3943 - val_categorical_accuracy: 0.5320
Epoch 106/110
73/73

In [19]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [20]:
# Xception:
run_xception()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Xception run begun at Thursday, 2017 October 19, 5:09 PM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 7...
Epoch 1/7
73/73 [==============================] - 285s - loss: 1.6108 - categorical_accuracy: 0.4238 - val_loss: 2.0718 - val_categorical_accuracy: 0.3852
Epoch 2/7
73/73 [==============================] - 282s - loss: 1.5077 - categorical_accuracy: 0.4724 - val_loss: 1.8523 - val_categorical_accuracy: 0.4055
Epoch 3/7
73/73 [==============================] - 282s - loss: 1.5083 - categorical_accuracy: 0.4670 - val_loss: 1.5743 - val_categorical_accuracy: 0.4539
Epoch 4/7
73/73 [==============================] - 280s - loss: 1.4882 - categorical_accuracy: 0.4714 - val_loss: 1.5026 - val_categorical_accuracy: 0.4766
Epoch 5/7
73/73 [===========================

73/73 [==============================] - 386s - loss: 1.2868 - categorical_accuracy: 0.5514 - val_loss: 1.4103 - val_categorical_accuracy: 0.5180
Epoch 98/100
73/73 [==============================] - 384s - loss: 1.2848 - categorical_accuracy: 0.5519 - val_loss: 1.4106 - val_categorical_accuracy: 0.5180
Epoch 99/100
73/73 [==============================] - 386s - loss: 1.2960 - categorical_accuracy: 0.5490 - val_loss: 1.4110 - val_categorical_accuracy: 0.5211
Epoch 100/100
73/73 [==============================] - 386s - loss: 1.2885 - categorical_accuracy: 0.5522 - val_loss: 1.4097 - val_categorical_accuracy: 0.5234


Further training (refining convolutional blocks, starting with
	layer 75)...

Training for epochs 101 to 107...
Epoch 101/107
73/73 [==============================] - 412s - loss: 1.2820 - categorical_accuracy: 0.5534 - val_loss: 1.4088 - val_categorical_accuracy: 0.5227
Epoch 102/107
73/73 [==============================] - 408s - loss: 1.2595 - categorical_accuracy: 0.5

In [21]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [22]:
# InceptionV3:
run_inception_v3()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
Inception V3 run begun at Friday, 2017 October 20, 4:40 AM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 10...
Epoch 1/10
73/73 [==============================] - 186s - loss: 1.6805 - categorical_accuracy: 0.3983 - val_loss: 3.6134 - val_categorical_accuracy: 0.1938
Epoch 2/10
73/73 [==============================] - 180s - loss: 1.5790 - categorical_accuracy: 0.4361 - val_loss: 1.9119 - val_categorical_accuracy: 0.3117
Epoch 3/10
73/73 [==============================] - 180s - loss: 1.5715 - categorical_accuracy: 0.4350 - val_loss: 1.6578 - val_categorical_accuracy: 0.4094
Epoch 4/10
73/73 [==============================] - 180s - loss: 1.5504 - categorical_accuracy: 0.4449 - val_loss: 1.5751 - val_categorical_accuracy: 0.4484
Epoch 5/10
73/73 [===================

73/73 [==============================] - 228s - loss: 1.2928 - categorical_accuracy: 0.5425 - val_loss: 1.4183 - val_categorical_accuracy: 0.5109
Epoch 98/100
73/73 [==============================] - 227s - loss: 1.2785 - categorical_accuracy: 0.5521 - val_loss: 1.4177 - val_categorical_accuracy: 0.5133
Epoch 99/100
73/73 [==============================] - 227s - loss: 1.2820 - categorical_accuracy: 0.5428 - val_loss: 1.4175 - val_categorical_accuracy: 0.5109
Epoch 100/100
73/73 [==============================] - 228s - loss: 1.2881 - categorical_accuracy: 0.5481 - val_loss: 1.4192 - val_categorical_accuracy: 0.5086


Further training (refining convolutional blocks, starting with
	layer 187)...

Training for epochs 101 to 110...
Epoch 101/110
73/73 [==============================] - 240s - loss: 1.2437 - categorical_accuracy: 0.5620 - val_loss: 1.4182 - val_categorical_accuracy: 0.5133
Epoch 102/110
73/73 [==============================] - 233s - loss: 1.2335 - categorical_accuracy: 0.

In [23]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [24]:
# ResNet50:
run_resnet50()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
ResNet50 run begun at Friday, 2017 October 20, 11:40 AM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 7...
Epoch 1/7
73/73 [==============================] - 226s - loss: 1.5162 - categorical_accuracy: 0.4662 - val_loss: 2.6943 - val_categorical_accuracy: 0.2852
Epoch 2/7
73/73 [==============================] - 223s - loss: 1.4102 - categorical_accuracy: 0.5054 - val_loss: 1.6660 - val_categorical_accuracy: 0.4109
Epoch 3/7
73/73 [==============================] - 222s - loss: 1.3872 - categorical_accuracy: 0.5210 - val_loss: 1.4792 - val_categorical_accuracy: 0.5000
Epoch 4/7
73/73 [==============================] - 222s - loss: 1.3853 - categorical_accuracy: 0.5182 - val_loss: 1.4241 - val_categorical_accuracy: 0.5281
Epoch 5/7
73/73 [============================

73/73 [==============================] - 272s - loss: 1.1322 - categorical_accuracy: 0.6094 - val_loss: 1.3491 - val_categorical_accuracy: 0.5469
Epoch 98/100
73/73 [==============================] - 270s - loss: 1.1515 - categorical_accuracy: 0.6032 - val_loss: 1.3488 - val_categorical_accuracy: 0.5437
Epoch 99/100
73/73 [==============================] - 271s - loss: 1.1381 - categorical_accuracy: 0.6088 - val_loss: 1.3496 - val_categorical_accuracy: 0.5422
Epoch 100/100
73/73 [==============================] - 271s - loss: 1.1362 - categorical_accuracy: 0.6093 - val_loss: 1.3487 - val_categorical_accuracy: 0.5469


Further training (refining convolutional blocks, starting with
	layer 119)...

Training for epochs 101 to 107...
Epoch 101/107
73/73 [==============================] - 287s - loss: 1.1094 - categorical_accuracy: 0.6183 - val_loss: 1.3496 - val_categorical_accuracy: 0.5469
Epoch 102/107
73/73 [==============================] - 282s - loss: 1.1053 - categorical_accuracy: 0.

In [25]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [12]:
# VGG16:
run_vgg16()
Audio(url=audio_file, autoplay=True)

Best optimizer (0.01, 0.02, 0.85, 1e-10) loaded from file.
Using optimizer (0.01, 0.02, 0.85, 1e-10)...
VGG16 run begun at Sunday, 2017 October 22, 5:34 AM.
	[20 epochs (x6 passes) on extended FMA on GPU takes
	unknown (no similar runs found).]

First-round training (training the classifier)...

Training for epochs 1 to 3...
Epoch 1/3
73/73 [==============================] - 364s - loss: 1.5456 - categorical_accuracy: 0.4560 - val_loss: 1.9779 - val_categorical_accuracy: 0.4820
Epoch 2/3
73/73 [==============================] - 364s - loss: 1.4396 - categorical_accuracy: 0.4991 - val_loss: 1.5883 - val_categorical_accuracy: 0.4617
Epoch 3/3
73/73 [==============================] - 364s - loss: 1.4133 - categorical_accuracy: 0.5094 - val_loss: 1.4578 - val_categorical_accuracy: 0.4844

Training for epochs 4 to 6...
Epoch 4/6
73/73 [==============================] - 363s - loss: 1.4040 - categorical_accuracy: 0.5117 - val_loss: 1.5342 - val_categorical_accuracy: 0.4789
Epoch 5/6
73/73 [=

73/73 [==============================] - 566s - loss: 1.0268 - categorical_accuracy: 0.6535 - val_loss: 1.3800 - val_categorical_accuracy: 0.5406
Epoch 94/95
73/73 [==============================] - 563s - loss: 0.9832 - categorical_accuracy: 0.6707 - val_loss: 1.3812 - val_categorical_accuracy: 0.5367
Epoch 95/95
73/73 [==============================] - 564s - loss: 1.0237 - categorical_accuracy: 0.6480 - val_loss: 1.2899 - val_categorical_accuracy: 0.5734

Training for epochs 96 to 98...
Epoch 96/98
73/73 [==============================] - 564s - loss: 1.0100 - categorical_accuracy: 0.6581 - val_loss: 1.3179 - val_categorical_accuracy: 0.5586
Epoch 97/98
73/73 [==============================] - 563s - loss: 0.9612 - categorical_accuracy: 0.6752 - val_loss: 1.3812 - val_categorical_accuracy: 0.5539
Epoch 98/98
73/73 [==============================] - 562s - loss: 1.0088 - categorical_accuracy: 0.6510 - val_loss: 1.2936 - val_categorical_accuracy: 0.5734

Training for epochs 99 to 100.

In [13]:
# Must be after Alert() call and in a separate cell for both audio and pop-up; sleep 
# allows the audio to play before the pop-up alters HTML output on the page:
delayed_popup()

In [14]:
# Back up the results dataframes
import shutil

for key in ["fma_results_name", "crossval_results_name"]:
    src = os.path.join("saved_objects", "{}.pkl".format(param_dict[key])) 
    dst = os.path.join("saved_object_backups", 
                       "{}-{}.pkl.bak".format(param_dict[key],
                                              timer.datetimepath()))
    directory = os.path.dirname(dst)
    if not os.path.exists(directory):
        os.makedirs(directory)
    #shutil.copyfile(src, dst)

    print ("Backed up '{}' to\n\t'{}'.\n".format(src, dst))

Backed up 'saved_objects/fma_results_gpu.pkl' to
	'saved_object_backups/fma_results_gpu-2017-10-22+2125.pkl.bak'.

Backed up 'saved_objects/crossval_results_gpu.pkl' to
	'saved_object_backups/crossval_results_gpu-2017-10-22+2125.pkl.bak'.



## Test Set Evaluation

After training, all trained models were evaluated on the test split of the relevant dataset. Because all final weights are saved during training, this can be performed separately from the training phase.

In [31]:
indices_of_runs = ut.load_obj("indices_of_runs")
try:
    fma_results = ut.load_obj("fma_results_numbercrunch")
except:
    fma_results = ut.load_obj("fma_results_gpu")
    files = os.listdir("saved_weights")
    
    # Set the earliest time for which runs are valid (i.e., exclude earlier debugging
    # runs):
    deftime = datetime.strptime('2017-10-15+0000', '%Y-%m-%d+%H%M') 

    for item in indices_of_runs:
        latest_time = deftime
        latest_weights = None
        model_key = fma_results.loc[item]["Model"]
        run_key = cku.formatted(cku.recover_run_key(fma_results.loc[item]))
        for file in files:
            if file.startswith(model_key):
                if run_key in file:
                    time_segment = file[-18:-3]
                    time_parsed = datetime.strptime(time_segment, '%Y-%m-%d+%H%M')
                    if time_parsed > latest_time:
                        latest_time = time_parsed
                        latest_weights = file
            fma_results.loc[item,"Weights File"] = latest_weights

    fma_results["(Test Predictions, inclass)"] = pd.Series(index=fma_results.index, 
                                                           dtype = "object")
    fma_results["(Test Predictions, correct)"] = pd.Series(index=fma_results.index, 
                                                           dtype = "object")
    fma_results["(Test Predictions, incorrect_thought_was)"] = pd.Series(
                                                           index=fma_results.index, 
                                                           dtype = "object")
    fma_results["(Test Predictions, incorrect_thought_wasnt)"] = pd.Series(
                                                           index=fma_results.index, 
                                                           dtype = "object")
ut.save_obj(fma_results,"fma_results_numbercrunch")
    
display(fma_results.loc[indices_of_runs])

,Run Started,Source Processor,Source,Pass Epochs,Batch Size,Steps Per Epoch,Validation Steps Per Epoch,Data Augmentation Factor,Data Set Size,Wavelet,...,Final Validation Accuracy,Training Loss History,Validation Loss History,Training Accuracy History,Validation Accuracy History,Weights File,"(Test Predictions, inclass)","(Test Predictions, correct)","(Test Predictions, incorrect_thought_was)","(Test Predictions, incorrect_thought_wasnt)"
450,"Monday, 2017 October 16, 10:30 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.00,small,dwt,...,0.320000,"[1.82380256891, 1.6008654356, 1.40744186163, 1...","[1.8220331955, 1.66511300087, 1.65917759418, 1...","[0.3171875, 0.41875, 0.50671875, 0.6015625, 0....","[0.36375, 0.36875, 0.36375, 0.395, 0.36, 0.375...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[34.0, 21.0, 19.0, 47.0, 40.0, 29.0, 18.0, 33.0]","[76.0, 98.0, 84.0, 52.0, 40.0, 67.0, 84.0, 58.0]","[66.0, 79.0, 81.0, 53.0, 60.0, 71.0, 82.0, 67.0]"
465,"Wednesday, 2017 October 18, 4:35 AM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.33,small,dwt,...,0.433750,"[1.82024466991, 1.71069197178, 1.67135448933, ...","[1.814739151, 1.67947849274, 1.63117892742, 1....","[0.3175, 0.37109375, 0.38828125, 0.40578125, 0...","[0.35, 0.375, 0.39, 0.405, 0.39875, 0.41125, 0...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[51.0, 26.0, 30.0, 70.0, 40.0, 36.0, 13.0, 49.0]","[52.0, 76.0, 80.0, 65.0, 42.0, 74.0, 50.0, 46.0]","[49.0, 74.0, 70.0, 30.0, 60.0, 64.0, 87.0, 51.0]"
461,"Tuesday, 2017 October 17, 11:16 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.50,small,dwt,...,0.440000,"[1.82721065044, 1.71751484394, 1.67218647718, ...","[1.80424007416, 1.68645466328, 1.64975505829, ...","[0.31625, 0.364375, 0.3853125, 0.39640625, 0.4...","[0.36125, 0.38375, 0.375, 0.4025, 0.3975, 0.40...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[43.0, 25.0, 35.0, 68.0, 38.0, 35.0, 19.0, 48.0]","[47.0, 72.0, 87.0, 67.0, 44.0, 74.0, 49.0, 49.0]","[57.0, 75.0, 65.0, 32.0, 62.0, 65.0, 81.0, 52.0]"
456,"Tuesday, 2017 October 17, 9:27 AM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.66,small,dwt,...,0.415000,"[1.823697927, 1.71283866167, 1.68205006361, 1....","[1.83449396133, 1.70822524071, 1.63681973457, ...","[0.31234375, 0.37171875, 0.38453125, 0.4017187...","[0.35125, 0.3775, 0.38375, 0.405, 0.405, 0.398...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[46.0, 24.0, 35.0, 66.0, 38.0, 39.0, 20.0, 45.0]","[54.0, 77.0, 81.0, 59.0, 44.0, 76.0, 49.0, 47.0]","[54.0, 76.0, 65.0, 34.0, 62.0, 61.0, 80.0, 55.0]"
469,"Wednesday, 2017 October 18, 11:51 AM",gpu,7.257224e+13,20.0,128.0,73.0,10.0,0.00,extended,dwt,...,0.493750,"[1.58588705814, 1.46814873937, 1.44014323901, ...","[1.56640315056, 1.46727433205, 1.43934185505, ...","[0.434075342466, 0.494220890411, 0.49732448630...","[0.49140625, 0.5, 0.50625, 0.4921875, 0.511718...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x2...,"[839.0, 1085.0, 299.0, 323.0, 309.0, 128.0, 20...","[454.0, 442.0, 29.0, 148.0, 11.0, 5.0, 5.0, 11...","[568.0, 499.0, 97.0, 220.0, 15.0, 14.0, 97.0, ...","[385.0, 643.0, 270.0, 175.0, 298.0, 123.0, 199..."
472,"Thursday, 2017 October 19, 12:47 PM",gpu,7.257224e+13,20.0,128.0,73.0,10.0,0.33,extended,dwt,...,0.539062,"[1.58695063199, 1.47280326608, 1.44755986945, ...","[1.54402760267, 1.48328278065, 1.45242022276, ...","[0.433112157534, 0.488976883562, 0.48919092465...","[0.49453125, 0.48984375, 0.496875, 0.49375, 0....",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x2...,"[839.0, 1085.0, 299.0, 323.0, 309.0, 128.0, 20...","[601.0, 397.0, 47.0, 171.0, 3.0, 7.0, 3.0, 116...","[822.0, 390.0, 108.0, 141.0, 2.0, 10.0, 26.0, ...","[238.0, 688.0, 252.0, 152.0, 306.0, 121.0, 201..."
451,"Monday, 2017 October 16, 10:42 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.00,small,dwt,...,0.362500,"[1.90809360027, 1.68670711

In [32]:
test_gens = {}
for size in ["small", "extended"]:
    param_dict["which_size"] = size
    # Set up path
    param_dict["img_dir"] = os.path.join("data",
                                 os.path.join("fma_images",
                                      os.path.join("byclass", 
                                           os.path.join(param_dict["which_size"], 
                                                        param_dict["which_wavelet"])
                                          )
                                     )
                                )
    g = cku.set_up_generators(param_dict)
    test_gens[size] = g[2]
    print("\n")


Creating generators with batch size 4...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_small_dwt_stats.npz'.

Found 6400 images belonging to 8 classes.
Found 800 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


Creating generators with batch size 4...
Loading mean and standard deviation for the training set from file 'saved_objects/fma_extended_dwt_stats.npz'.

Found 37316 images belonging to 8 classes.
Found 4350 images belonging to 8 classes.
Found 4651 images belonging to 8 classes.




In [33]:
def generator_with_true_classes(model, generator):
    while True:
        x, y = generator.next()
        try:
            y_pred = model.predict(x)
        except:
            y_pred = np.ndarray((0,0))
        yield x, y_pred, y

In [34]:
def predict_model(num_classes, test_gens, fma_results, item, models_printed):
    # Set up the generators
    item_data_size = fma_results.loc[item, "Data Set Size"]
    test_gen = test_gens[item_data_size]
    test_set_size = 800 if item_data_size == "small" else 4651
        
    if (fma_results.loc[item, "Model"] == "fcnn"):
        # Set up the input to accept FMA images:
        inp = keras.layers.Input(shape=(256,256,3,))

        # Add a flatten layer to make the input play nicely with these non-convolutional 
        # layers:
        x = keras.layers.Flatten()(inp)

        # Add a Flatten/Affine/BatchNorm/ReLU/Dropout/Affine-softmax-categorization block:
        predict = cku.stack_two_layer_block(param_dict, x)
        
        # Construct the model:
        model = keras.models.Model(inp,predict)
    else:
        if (fma_results.loc[item, "Model"] == "xception"):
            model_class = keras.applications.xception.Xception
        elif (fma_results.loc[item, "Model"] == "inception_v3"):
            model_class = keras.applications.inception_v3.InceptionV3
        elif (fma_results.loc[item, "Model"] == "resnet50"):
            model_class = keras.applications.resnet50.ResNet50
        elif (fma_results.loc[item, "Model"] == "vgg16"):
            model_class = keras.applications.vgg16.VGG16
        else:
            print("Unrecognized: ", fma_results.loc[item, "Model"])
            raise ValueError
        
        # Set up the model
        basemodel = model_class(include_top=False, 
                                input_shape=param_dict["mean_img"].shape)
        x = basemodel.output
        
        # Add a global spatial average pooling layer at the output:
        x = keras.layers.GlobalAveragePooling2D()(x)

        # Add Affine/BatchNorm/ReLU/Dropout/Affine-softmax-categorization block:
        predict = cku.stack_two_layer_block(param_dict, x)

        # Now make the model:
        model = keras.models.Model(basemodel.input, predict)


    # Describe the model, if one of this type hasn't been printed already - this code is
    # included here and excluded elsewhere to prevent the notebook from getting overly 
    # cluttered:
    if not models_printed[fma_results.loc[item, "Model"]]:
        print("Model type: ",fma_results.loc[item, "Model"])
        i = 0
        for layer in model.layers:
            print("\t{:d}:\t{:s}".format(i,layer.name))
            i += 1
        print()
    
        models_printed[fma_results.loc[item, "Model"]] = True
    
    # Load the weights:
    model.load_weights(os.path.join("saved_weights", 
                                    fma_results.loc[item, "Weights File"]))
    
    # Predict:
    # Flow data with correct labels, http://bit.ly/2iCudEJ
    num_batches = 0
    max_batches = (int)(np.ceil(test_set_size/test_gen.batch_size))
    print("Max batches: ", max_batches)
    preds = np.ndarray((test_set_size, num_classes+1))
    start_idx = 0
    for x, y_pred, y_true in generator_with_true_classes(model, test_gen):
        # do something with data, eg. print it.
        end_idx = np.min([test_set_size, 
                          start_idx + y_true.shape[0], 
                          start_idx + y_pred.shape[0]])
        
        slice_len = np.min([test_set_size-start_idx, 
                            y_true.shape[0],
                            y_pred.shape[0]])
        
        preds[start_idx:(start_idx + slice_len), 0] = np.argmax(y_true)
        preds[start_idx:(start_idx + slice_len), 1:] = y_pred
        
        start_idx += test_gen.batch_size
        num_batches += 1
        if num_batches == max_batches:
             break
    
    # Do some number crunching
    correct_classes = preds[:,0]
    predicted_classes = np.argmax(preds[:,1:], axis = 1)

    inclass = np.zeros((param_dict["num_classes"],1))
    corr = np.zeros((param_dict["num_classes"],1))
    incorr_thought_was = np.zeros((param_dict["num_classes"],1))
    incorr_thought_wasnt = np.zeros((param_dict["num_classes"],1))
    for cls in np.arange(param_dict["num_classes"]):
        # How many items actually are in this class?
        inclass[cls] = np.sum(correct_classes == cls)

        # Correct:
        corr_idxes = np.argwhere(correct_classes == cls)
        corr[cls] = np.sum(correct_classes[corr_idxes] == 
                           predicted_classes[corr_idxes])

        # Incorrectly identified as this class:
        iden_idxes = np.argwhere(predicted_classes == cls) 
        # thought it was this class
        incorr_thought_was[cls] = np.sum(correct_classes[iden_idxes] != cls) 
        # but it wasn't

        # Incorrectly identified as not this class:
        iden_idxes = np.argwhere(predicted_classes != cls) 
        # thought it wasn't this class
        incorr_thought_wasnt[cls] = np.sum(correct_classes[iden_idxes] == cls) 
        # but it was

    fma_results.loc[item, "(Test Predictions, inclass)"] = inclass
    fma_results.loc[item, "(Test Predictions, correct)"] = corr
    fma_results.loc[item, "(Test Predictions, incorrect_thought_was)"] = (
                                                    incorr_thought_was)
    fma_results.loc[item, "(Test Predictions, incorrect_thought_wasnt)"] = (
                                                    incorr_thought_wasnt)

    return preds

In [35]:
models_printed = {}
for model in ["fcnn","xception","inception_v3","resnet50","vgg16"]:
    models_printed[model] = False
    
last_start_by = datetime(2017, 11, 8, 9, 45, 0)

for item in indices_of_runs:
    print()
    print(fma_results.loc[item, "Weights File"])
    if pd.isnull(fma_results["(Test Predictions, inclass)"]).loc[item]:
        print("Calculation started: ", timer.datetimestamp())
        preds = predict_model(param_dict["num_classes"], test_gens, fma_results, item,
                              models_printed)
        ut.save_obj(fma_results,"fma_results_numbercrunch")
        print("Shape of predictions: ", preds.shape)
        print("Calculation complete: ", timer.datetimestamp())
        print()
        
        if datetime.now() > last_start_by :
            break
    else:
        print("Skipping {:d}; already calculated...".format(item))
        

        
print("\n\n\nComplete!")


fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0xsmallxdwt_2017-10-16+2242.h5
Skipping 450; already calculated...

fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0.33xsmallxdwt_2017-10-18+0520.h5
Skipping 465; already calculated...

fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0.5xsmallxdwt_2017-10-18+0001.h5
Skipping 461; already calculated...

fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0.66xsmallxdwt_2017-10-17+1013.h5
Skipping 456; already calculated...

fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x20x128x73x10x0xextendedxdwt_2017-10-18+1311.h5
Skipping 469; already calculated...

fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x20x128x73x10x0.33xextendedxdwt_2017-10-19+1708.h5
Skipping 472; already calculated...

xception_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0xsmallxdwt_2017-10-17+0040.h5
Skipping 451; already calculated...

xception_0.01x0.02x0.85x1e-10_gpux72572239216642x5x128x50x7x0.33xsmallxdwt_2017-10-18+0718.h5
Skipping 466

In [36]:
display(fma_results.loc[indices_of_runs])
ut.save_obj(fma_results,"fma_results_numbercrunch")

,Run Started,Source Processor,Source,Pass Epochs,Batch Size,Steps Per Epoch,Validation Steps Per Epoch,Data Augmentation Factor,Data Set Size,Wavelet,...,Final Validation Accuracy,Training Loss History,Validation Loss History,Training Accuracy History,Validation Accuracy History,Weights File,"(Test Predictions, inclass)","(Test Predictions, correct)","(Test Predictions, incorrect_thought_was)","(Test Predictions, incorrect_thought_wasnt)"
450,"Monday, 2017 October 16, 10:30 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.00,small,dwt,...,0.320000,"[1.82380256891, 1.6008654356, 1.40744186163, 1...","[1.8220331955, 1.66511300087, 1.65917759418, 1...","[0.3171875, 0.41875, 0.50671875, 0.6015625, 0....","[0.36375, 0.36875, 0.36375, 0.395, 0.36, 0.375...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[34.0, 21.0, 19.0, 47.0, 40.0, 29.0, 18.0, 33.0]","[76.0, 98.0, 84.0, 52.0, 40.0, 67.0, 84.0, 58.0]","[66.0, 79.0, 81.0, 53.0, 60.0, 71.0, 82.0, 67.0]"
465,"Wednesday, 2017 October 18, 4:35 AM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.33,small,dwt,...,0.433750,"[1.82024466991, 1.71069197178, 1.67135448933, ...","[1.814739151, 1.67947849274, 1.63117892742, 1....","[0.3175, 0.37109375, 0.38828125, 0.40578125, 0...","[0.35, 0.375, 0.39, 0.405, 0.39875, 0.41125, 0...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[51.0, 26.0, 30.0, 70.0, 40.0, 36.0, 13.0, 49.0]","[52.0, 76.0, 80.0, 65.0, 42.0, 74.0, 50.0, 46.0]","[49.0, 74.0, 70.0, 30.0, 60.0, 64.0, 87.0, 51.0]"
461,"Tuesday, 2017 October 17, 11:16 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.50,small,dwt,...,0.440000,"[1.82721065044, 1.71751484394, 1.67218647718, ...","[1.80424007416, 1.68645466328, 1.64975505829, ...","[0.31625, 0.364375, 0.3853125, 0.39640625, 0.4...","[0.36125, 0.38375, 0.375, 0.4025, 0.3975, 0.40...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[43.0, 25.0, 35.0, 68.0, 38.0, 35.0, 19.0, 48.0]","[47.0, 72.0, 87.0, 67.0, 44.0, 74.0, 49.0, 49.0]","[57.0, 75.0, 65.0, 32.0, 62.0, 65.0, 81.0, 52.0]"
456,"Tuesday, 2017 October 17, 9:27 AM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.66,small,dwt,...,0.415000,"[1.823697927, 1.71283866167, 1.68205006361, 1....","[1.83449396133, 1.70822524071, 1.63681973457, ...","[0.31234375, 0.37171875, 0.38453125, 0.4017187...","[0.35125, 0.3775, 0.38375, 0.405, 0.405, 0.398...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x5...,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[46.0, 24.0, 35.0, 66.0, 38.0, 39.0, 20.0, 45.0]","[54.0, 77.0, 81.0, 59.0, 44.0, 76.0, 49.0, 47.0]","[54.0, 76.0, 65.0, 34.0, 62.0, 61.0, 80.0, 55.0]"
469,"Wednesday, 2017 October 18, 11:51 AM",gpu,7.257224e+13,20.0,128.0,73.0,10.0,0.00,extended,dwt,...,0.493750,"[1.58588705814, 1.46814873937, 1.44014323901, ...","[1.56640315056, 1.46727433205, 1.43934185505, ...","[0.434075342466, 0.494220890411, 0.49732448630...","[0.49140625, 0.5, 0.50625, 0.4921875, 0.511718...",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x2...,"[839.0, 1085.0, 299.0, 323.0, 309.0, 128.0, 20...","[454.0, 442.0, 29.0, 148.0, 11.0, 5.0, 5.0, 11...","[568.0, 499.0, 97.0, 220.0, 15.0, 14.0, 97.0, ...","[385.0, 643.0, 270.0, 175.0, 298.0, 123.0, 199..."
472,"Thursday, 2017 October 19, 12:47 PM",gpu,7.257224e+13,20.0,128.0,73.0,10.0,0.33,extended,dwt,...,0.539062,"[1.58695063199, 1.47280326608, 1.44755986945, ...","[1.54402760267, 1.48328278065, 1.45242022276, ...","[0.433112157534, 0.488976883562, 0.48919092465...","[0.49453125, 0.48984375, 0.496875, 0.49375, 0....",fcnn_0.01x0.02x0.85x1e-10_gpux72572239216642x2...,"[839.0, 1085.0, 299.0, 323.0, 309.0, 128.0, 20...","[601.0, 397.0, 47.0, 171.0, 3.0, 7.0, 3.0, 116...","[822.0, 390.0, 108.0, 141.0, 2.0, 10.0, 26.0, ...","[238.0, 688.0, 252.0, 152.0, 306.0, 121.0, 201..."
451,"Monday, 2017 October 16, 10:42 PM",gpu,7.257224e+13,5.0,128.0,50.0,7.0,0.00,small,dwt,...,0.362500,"[1.90809360027, 1.68670711

In [43]:
iterables_a = [["fcnn","inception_v3","xception","resnet50","vgg16"], 
               ["small"],
               [0, 0.33, 0.5, 0.66]]
iterables_b = [["fcnn","inception_v3","xception","resnet50","vgg16"], 
               ["extended"],
               [0, 0.33]]
hier_idx = pd.MultiIndex.from_product(iterables_a, names=['Model', 
                                                          'Data Set Size', 
                                                          'Data Augmentation Factor'])
hier_idx = hier_idx.append(pd.MultiIndex.from_product(iterables_b, names=['Model', 
                                                          'Data Set Size', 
                                                          'Data Augmentation Factor']))
iterables_cols = [["Overall"],
                  ["count", "correct", "correct_pct"]]
hier_cols = pd.MultiIndex.from_product(iterables_cols, names=['Class', 
                                                              'Statistic'])
iterables_cols = [param_dict["classes"],
                  ["in_class", 
                   "correct", "correct_pct", 
                   "incorrect_thought_was", "incorrect_thought_was_pct", 
                   "incorrect_thought_wasnt", "incorrect_thought_wasnt_pct"]]
hier_cols = hier_cols.append(pd.MultiIndex.from_product(iterables_cols, names=['Class', 
                                                              'Statistic']))
try:
    test_results = ut.load_obj("test_results")
except:
    test_results = pd.DataFrame(index = hier_idx, columns = hier_cols)

In [44]:
for item in indices_of_runs:
    model = fma_results.loc[item, "Model"]
    datasize = fma_results.loc[item, "Data Set Size"]
    daf = fma_results.loc[item, "Data Augmentation Factor"]
    row = ((test_results.index.get_level_values("Model") == model) & 
             (test_results.index.get_level_values("Data Set Size") == datasize) &
             (test_results.index.get_level_values("Data Augmentation Factor") == daf))
    try:
        tot_count = 0
        corr_count = 0
        for idx in np.arange(param_dict["num_classes"]):
            cls_name = param_dict["classes"][idx]
            count = fma_results.loc[item, "(Test Predictions, inclass)"][idx]
            tot_count += count

            # Count
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == "in_class")] = (
                        count
                 )
            # Correct vals:
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == "correct")] = (
                        fma_results.loc[item, "(Test Predictions, correct)"][idx]
                 )
            corr_count += fma_results.loc[item, "(Test Predictions, correct)"][idx]
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == "correct_pct")] = (
                        fma_results.loc[item, "(Test Predictions, correct)"][idx]/count
                 )
            # Incorrect, thought was vals:
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == 
                          "incorrect_thought_was")] = (
                        fma_results.loc[item, 
                            "(Test Predictions, incorrect_thought_was)"][idx]
                 )
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == 
                          "incorrect_thought_was_pct")] = (
                        fma_results.loc[item, 
                            "(Test Predictions, incorrect_thought_was)"][idx]/count
                 )

            # Incorrect, thought wasn't vals:
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == 
                          "incorrect_thought_wasnt")] = (
                        fma_results.loc[item, 
                            "(Test Predictions, incorrect_thought_wasnt)"][idx]
                 )
            test_results.loc[row,
               (test_results.columns.get_level_values("Class") == cls_name) &
                 (test_results.columns.get_level_values("Statistic") == 
                          "incorrect_thought_wasnt_pct")] = (
                        fma_results.loc[item, 
                            "(Test Predictions, incorrect_thought_wasnt)"][idx]/count
                 )
        # Model (row) statistics:
        test_results.loc[row,
               (test_results.columns.get_level_values("Class") == "Overall") &
                 (test_results.columns.get_level_values("Statistic") == "count")] = (
                        tot_count
                )
        test_results.loc[row,
               (test_results.columns.get_level_values("Class") == "Overall") &
                 (test_results.columns.get_level_values("Statistic") == "correct")] = (
                        corr_count
                )
        test_results.loc[row,
               (test_results.columns.get_level_values("Class") == "Overall") &
                 (test_results.columns.get_level_values("Statistic") == "correct_pct")] = (
                        corr_count/tot_count
                )
    except:
        pass

In [46]:
# # Show everything 
# display(test_results)

# Show a single example
test_results.loc[(test_results.index.get_level_values("Model") == "fcnn") & 
                 (test_results.index.get_level_values("Data Set Size") == "small") &
                 (test_results.index.get_level_values("Data Augmentation Factor") == 0)]

Class                                        Overall                      \
Statistic                                      count correct correct_pct   
Model Data Set Size Data Augmentation Factor                               
fcnn  small         0.0                          800     241     0.30125   

Class                                        electronic                      \
Statistic                                      in_class correct correct_pct   
Model Data Set Size Data Augmentation Factor                                  
fcnn  small         0.0                             100      34        0.34   

Class                                                               \
Statistic                                    incorrect_thought_was   
Model Data Set Size Data Augmentation Factor                         
fcnn  small         0.0                                         76   

Class                                                                   \
Statistic                                    incorrect_thought_was_pct   
Model Data Set Size Data Augmentation Factor                             
fcnn  small         0.0                                           0.76   

Class                                                                 \
Statistic                                    incorrect_thought_wasnt   
Model Data Set Size Data Augmentation Factor                           
fcnn  small         0.0                                           66   

Class                                                                     \
Statistic                                    incorrect_thought_wasnt_pct   
Model Data Set Size Data Augmentation Factor                               
fcnn  small         0.0                                             0.66   

Class                                                    ...              \
Statistic                                                ...               
Model Data Set Size Data Augmentation Factor             ...               
fcnn  small         0.0                                  ...               

Class                                                              pop  \
Statistic                                    incorrect_thought_was_pct   
Model Data Set Size Data Augmentation Factor                             
fcnn  small         0.0                                           0.84   

Class                                                                 \
Statistic                                    incorrect_thought_wasnt   
Model Data Set Size Data Augmentation Factor                           
fcnn  small         0.0                                           82   

Class                                                                     \
Statistic                                    incorrect_thought_wasnt_pct   
Model Data Set Size Data Augmentation Factor                               
fcnn  small         0.0                                             0.82   

Class                                            rock                      \
Statistic                                    in_class correct correct_pct   
Model Data Set Size Data Augmentation Factor                                
fcnn  small         0.0                           100      33        0.33   

Class                                                               \
Statistic                                    incorrect_thought_was   
Model Data Set Size Data Augmentation Factor                         
fcnn  small         0.0                                         58   

Class                                                                   \
Statistic                                    incorrect_thought_was_pct   
Model Data Set Size Data Augmentation Factor                             
fcnn  small         0.0                                           0.58   

Class                                                                 \
Statistic                                    incorr

In [47]:
ut.save_obj(test_results, "test_results")